In [1]:
def get_movies_since(year):
    import pandas as pd
    year_data = pd.read_csv('year.csv', on_bad_lines='skip', sep="\t")
    movie_data = year_data[year_data["titleType"].isin(["movie", "tvMovie"])]
    movie_data_with_year = movie_data[movie_data["startYear"] >= "1990"]
    movie_data_with_valid_year = movie_data_with_year[movie_data_with_year["startYear"] != "\\N"]
    movie_data_with_valid_year = movie_data_with_valid_year[["tconst", "startYear", "originalTitle"]]

    imdb_data = pd.read_csv("ratings.csv", sep="\t")

    year_vote_data = pd.merge(imdb_data, movie_data_with_valid_year, on="tconst", how="right")
    year_vote_data = year_vote_data[["tconst","startYear","averageRating","numVotes", "originalTitle"]]
    year_vote_data.columns = ["title id","Year","Rating","Votes", "Title"]
    year_vote_data["rating link"] = "https://www.imdb.com/title/"+year_vote_data["title id"]+"/ratings/?ref_=tt_ov_rt"
    year_vote_data = year_vote_data.sort_values(by="title id").reset_index(drop=True)
    year_vote_data.to_csv("movies_since_"+"1990"+".csv", index=False)

    return year_vote_data
dataset = get_movies_since("1990")

/tmp/ipykernel_5317/2064243291.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  year_data = pd.read_csv('year.csv', on_bad_lines='skip', sep="\t")


In [2]:
import pandas as pd
dataset = pd.read_csv("movies_since_1990.csv")
dataset.head()

,title id,Year,Rating,Votes,Title,rating link
0,tt0011801,2019,NaN,NaN,Tötet nicht mehr,https://www.imdb.com/title/tt0011801/ratings/?...
1,tt0013274,2021,6.4,36.0,Istoriya grazhdanskoy voyny,https://www.imdb.com/title/tt0013274/ratings/?...
2,tt0015414,2000,5.3,16.0,La tierra de los toros,https://www.imdb.com/title/tt0015414/ratings/?...
3,tt0015724,1993,6.1,26.0,Dama de noche,https://www.imdb.com/title/tt0015724/ratings/?...
4,tt0035423,2001,6.4,84785.0,Kate & Leopold,https://www.imdb.com/title/tt0035423/ratings/?...


## extracting vote data

In [1]:
def get_votes(link, vote_threshold=10000):
    import requests, re
    request_text = requests.get(link).text
    if "No Ratings Available" in request_text or "Well, what if there is no webpage?" in request_text or int(((request_text[request_text.index("IMDb users have given a ")-15:request_text.index("IMDb users have given a ")-1].strip()).replace(",",""))) < vote_threshold:
        return [0,0,0,0,0,0,0,0,0,0], 0.0

    star_votes = re.findall("<div class=\"leftAligned\">(.*?)</div>", request_text)
    rating = re.findall("span class=\"ipl-rating-star__rating\">(.*?)</span>", request_text)
    stars = [int(vote.replace(",","")) for vote in star_votes[1:11]]
    
    return stars, float(rating[0])

In [4]:
# 1 second for a link
stars, rating = get_votes("https://www.imdb.com/title/tt0068646/ratings/?ref_=tt_ov_rt", vote_threshold=10000)

In [2]:
def fill_dataset(dataset, start=0, end=0, zip=1):
    for i in range(start, end):
        try:
            stars, rating = get_votes(dataset.iloc[i]["rating link"])
            dataset.loc[i,["1","2","3","4","5","6","7","8","9","10"]] = stars
            dataset.loc[i,["Rating","Votes"]] = rating, sum(stars)
        except:
            pass
        if i % 100 == 0:
            print(i,". movie done between ",start,"-",end, sep="")

    if zip:
        print(end,". movie done between ",start,"-",end, sep="")
        dataset.to_csv("movie_data_at_"+str(start)+"_"+str(end)+".csv", index=False)
    return dataset

In [7]:
import pandas as pd
dataset = pd.read_csv("movie_data_at_5000_6000.csv")
dataset = fill_dataset(dataset, start=6000, end=7000, zip=1)

5000. movie done between 5000-6000
5100. movie done between 5000-6000
5200. movie done between 5000-6000
5300. movie done between 5000-6000
5400. movie done between 5000-6000
5500. movie done between 5000-6000
5600. movie done between 5000-6000
5700. movie done between 5000-6000
5800. movie done between 5000-6000
5900. movie done between 5000-6000
6000. movie done between 5000-6000
